In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random as rnd

#Rule for this notebook: If something is commented out, don't touch it unless you know 100% what it does - kai.

In [2]:
Local_dir = "K:\\Users\KJR\\Documents\\CSVs\\DATA3001\\" #replace however you need to
Event_list = pd.read_csv(Local_dir + "2019-2021_EventData.csv")
match_list = pd.read_csv(Local_dir + "2019-2021_MatchListing.csv")

In [87]:
Position_list = Event_list[Event_list['Points'] > 0]
Position_list.to_csv('Point_Event_list.csv')
#Position_list = Position_list[Position_list['SeasonId'] == 2019]
Scoring_indicies = Position_list.index
Scoring_indicies

Int64Index([   3850,    6226,    6635,    6652,    7961,    7972,    9204,
               9213,   11026,   12684,
            ...
            3385142, 3385554, 3386564, 3388581, 3388590, 3392810, 3392821,
            3393723, 3394249, 3394260],
           dtype='int64', length=4183)

In [88]:
rnd.seed(3) #Please use same seed
samp_size = 4183
Play_sample = rnd.sample(list(Scoring_indicies), samp_size)
Play_sample.sort()
plays = Event_list[Event_list.index.isin(Play_sample)]

In [89]:
i = 0
prev_plays = 100
for x in Play_sample:
    if i == samp_size: #Small check as play_sample is infinitely increasing
        break;
    for y in np.arange(x-prev_plays, x, 1):
        Play_sample.append(y)
    i += 1
Play_sample.sort()
print(len(Play_sample))

422483


In [90]:
sample = Event_list[Event_list.index.isin(Play_sample)]
Tallies_all = pd.DataFrame(columns = sample['EventCode'].unique())

In [91]:
for x in np.arange(0,samp_size,1):
    small_sample = sample.iloc[x*(prev_plays+1)+1:x*(prev_plays+1)+prev_plays] #Grabs the plays based on the prev_plays variable, tallies them up to the next position scoring play
    count = small_sample['EventCode'].value_counts()
    Tallies = pd.DataFrame(count)
    Tallies_all = Tallies_all.append(Tallies.transpose())
Tallies_all.index = plays.index



In [92]:
NoOpponents = pd.read_csv('NoOppEvents.csv')

NoOppCodes = NoOpponents['Code'].tolist()

NoOppCodes

for col in Tallies_all.columns:
    if col not in NoOppCodes:
        Tallies_all = Tallies_all.drop(col, axis = 1)
Tallies_all.columns

Index(['PBNS', 'RCVS', 'RSPS', 'MACS', 'MACD', 'HTPS', 'MBCS', 'MBCD', 'OADS',
       'TKES',
       ...
       'VCHS', 'CVMS', 'P3LS', 'RINS', 'RCAS', 'RKTS', 'SBKS', 'TTSD', 'I10S',
       'RSLS'],
      dtype='object', length=183)

In [93]:
Tallies_all = Tallies_all.fillna(0)
Tackles = ['TKES', 'REFT', 'TK1S', 'TKOS', 'FLPS', 'PFTA', 'TKSS', 'TKPS', 'TTSS', 'TKTS']
Passes = ['PDOS', 'PSOS', 'PSFS', 'DPLS', 'P1OS', 'DPRS', 'P2OS', 'P1FS', 'P2FS', 'LAPS', 'PFWS', 'PDFS', 'P3OS', 'P3FS', 'PSLS', 'PDLS', 'P2LS', 'P3LS', 'P1LS', 'PSKS']
Ruck_Infringements = ['DOPS', 'RINS']

Tallies_all['Tackle_sum'] = 0

for col in Tackles:
    Tallies_all['Tackle_sum'] = Tallies_all['Tackle_sum'] + Tallies_all[col]
print(Tallies_all['Tackle_sum'])

Tallies_all['Pass_sum'] = 0

for col in Passes:
    Tallies_all['Pass_sum'] = Tallies_all['Pass_sum'] + Tallies_all[col]
print(Tallies_all['Pass_sum'])

Tallies_all['RuckInfs'] = 0
for col in Ruck_Infringements:
    Tallies_all['RuckInfs'] = Tallies_all['RuckInfs'] + Tallies_all[col]
    
for col in Tallies_all.columns:
    if col  in Tackles:
        Tallies_all = Tallies_all.drop(col, axis = 1)
for col in Tallies_all.columns:
    if col  in Passes:
        Tallies_all = Tallies_all.drop(col, axis = 1)
for col in Tallies_all.columns:
    if col  in Ruck_Infringements:
        Tallies_all = Tallies_all.drop(col, axis = 1)

Tallies_all.columns

3850       11
6226        7
6635       11
6652        6
7961        8
           ..
3392810     0
3392821     0
3393723     0
3394249     0
3394260     0
Name: Tackle_sum, Length: 4183, dtype: int64
3850       0
6226       0
6635       0
6652       3
7961       1
          ..
3392810    0
3392821    0
3393723    0
3394249    0
3394260    0
Name: Pass_sum, Length: 4183, dtype: int64


Index(['PBNS', 'RCVS', 'RSPS', 'MACS', 'MACD', 'HTPS', 'MBCS', 'MBCD', 'OADS',
       'TKED',
       ...
       'CVMS', 'RCAS', 'RKTS', 'SBKS', 'TTSD', 'I10S', 'RSLS', 'Tackle_sum',
       'Pass_sum', 'RuckInfs'],
      dtype='object', length=154)

In [94]:
for col in Tallies_all.columns:
    if (np.sum(Tallies_all[col]) < 4183):
        Tallies_all.drop(col,inplace=True,axis=1)

In [107]:
sample = pd.concat([plays,Tallies_all], axis=1)
Unnecessary_Events = ['VenueId','Team A Name', 'Team B Name', 'SeriesId', 'WeatherConditionName',
                      'SeriesName', 'Jumper', 'VenueId', 'Qualifier1Name', 'Qualifier1', 'Qualifier2Name', 'Qualifier2',
                      'Qualifier3Name', 'Qualifier3', 'Qualifier4Name', 'Qualifier4', 'Qualifier5Name', 'Qualifier5',
                      'Qualifier6Name', 'Qualifier6', 'Qualifier7Name', 'Qualifier7',
                      'Qualifier8Name', 'Qualifier8', 'DurationSecs', 'DistanceMs', 'OfficialId', 'Team A id', 'Team B ID',
                      'Club Name', 'Opposition Id', 'InPossessionClub Id', 'InPossession Player Id', 'SeqNumber'
                     , 'ElapsedSecs', 'GameSecs', 'ChannelPhysical', 'SectionPhysical', 'ChannelPlayer', 'ZonePhysical', 'XmPlayer'
                     , 'Player Id', 'RunOn', 'EventName', 'YmPhysical', 'Club Id', 'TeamAScore', 'TeamBScore', 'MatchId', 'XmPhysical', 'SectionPhysical', 'XmPossession',
                     'SectionPlayer', 'YmPossession', 'ZonePossession', 'SectionPossession', 'YmPlayer', 'ZonePlayer', 'ElapsedMillisecs', 'ElapsedMins', 'GameMins', 'Set', 'Score', 'OppScore'
                     , 'PossessionSecs', 'OppPossessionSecs'] #So far, will trim as we get further along project
sample = sample.drop(Unnecessary_Events, axis=1)
sample

,SeasonId,RoundId,WeatherConditionId,PositionId,Captain,Half,Tackle,EventCode,Points,ChannelPossession,...,MACD,MBCS,MBCD,OADS,TKED,MKRS,MKRD,TRCS,Tackle_sum,Pass_sum
3850,2019,1,4,59.0,Y,1,0,PGOK,2.0,R,...,8,3,8,3,3,6,6,0,11,0
6226,2019,1,4,56.0,N,2,0,PGOK,2.0,C,...,8,3,8,4,2,4,4,0,7,0
6635,2019,1,4,61.0,N,2,6,TRY,4.0,C,...,8,3,8,3,3,6,6,2,11,0
6652,2019,1,4,56.0,N,2,0,CVOK,2.0,C,...,4,2,4,1,2,4,4,0,6,3
7961,2019,1,4,64.0,N,2,3,TRY,4.0,C,...,4,2,4,2,3,5,5,2,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3392810,2021,25,2,53.0,N,2,0,TRY,4.0,R,...,0,0,0,0,0,0,0,0,0,0
3392821,2021,25,2,59.0,N,2,0,CVOK,2.0,R,...,0,0,0,0,0,0,0,0,0,0
3393723,2021,25,2,53.0,N,2,3,TRY,4.0,SR,...,0,0,0,0,0,0,0,0,0,0
3394249,2021,25,2,57.0,N,2,5,TRY,4.0,SR,...,0,0,0,0,0,0,0,0,0,0


In [108]:
ScoreCodes = pd.get_dummies(sample['EventCode'])
HalfCodes = pd.get_dummies(sample['Half'])
PositionCodes = pd.get_dummies(sample['ChannelPossession'])
WeatherCodes = pd.get_dummies(sample['WeatherConditionId'])

ScoreCodes.columns = 'Goal_' + ScoreCodes.columns
for col in HalfCodes.columns:
    col1 = 'Half_' + str(col)
    HalfCodes = HalfCodes.rename({col:col1}, axis = 1)
for col in WeatherCodes.columns:
    col1 = 'Weather_' + str(col)
    WeatherCodes = WeatherCodes.rename({col:col1}, axis = 1)
sample = pd.concat([sample, PositionCodes, ScoreCodes, WeatherCodes, HalfCodes], axis = 1)
sample = sample.drop(['EventCode', 'ChannelPossession', 'WeatherConditionId', 'Half'], axis = 1)

back = [53,54,55,56,57]
mid = [58,59,65]
front = [60,61,62,63,64]

sample.loc[sample['Captain'] == 'Y', 'Captain'] = 1
sample.loc[sample['Captain'] == 'N', 'Captain'] = 0
for i in back:
    sample.loc[sample['PositionId'] == i, 'PositionId'] = 2

for j in mid:
    sample.loc[sample['PositionId'] == j, 'PositionId'] = 1
    
for k in front:
    sample.loc[sample['PositionId'] == k, 'PositionId'] = 0



sample = sample.fillna(0)
sample.to_csv('sample_all.csv')

In [109]:
sample

,SeasonId,RoundId,PositionId,Captain,Tackle,Points,TotalPossessionSecs,0_20_ElapsedSecs,20_Half_ElapsedSecs,Half_20_ElapsedSecs,...,Goal_PGOK,Goal_TRY,Weather_2,Weather_3,Weather_4,Weather_1040,Half_1,Half_2,Half_3,Half_4
3850,2019,1,1.0,1,0,2.0,1142.02,353040,554520,484440,...,1,0,0,0,1,0,1,0,0,0
6226,2019,1,2.0,0,0,2.0,1855.64,690280,776120,793120,...,1,0,0,0,1,0,0,1,0,0
6635,2019,1,0.0,0,6,4.0,1982.58,716640,806120,907400,...,0,1,0,0,1,0,0,1,0,0
6652,2019,1,2.0,0,0,2.0,1982.66,716640,806120,907400,...,0,0,0,0,1,0,0,1,0,0
7961,2019,1,0.0,0,3,4.0,2368.24,1094120,1026040,1041240,...,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3392810,2021,25,2.0,0,0,4.0,3115.62,991760,1364600,1388760,...,0,1,1,0,0,0,0,1,0,0
3392821,2021,25,1.0,0,0,2.0,3115.70,991760,1364600,1388760,...,0,0,1,0,0,0,0,1,0,0
3393723,2021,25,2.0,0,3,4.0,3389.56,1051840,1521320,1485960,...,0,1,1,0,0,0,0,1,0,0
3394249,2021,25,2.0,0,5,4.0,3557.92,1095800,1701760,1616800,...,0,1,1,0,0,0,0,1,0,0


In [98]:
sample = sample.drop(sample[sample['PositionId'] == 66].index)
sample

sample.to_csv('sample_all.csv')

In [99]:
newsample = Position_list[['SeasonId','RoundId', 'PositionId', 'Points', 'MatchId', 'EventCode']]

In [100]:
back = [53,54,55,56,57]
mid = [58,59,65,66]
front = [60,61,62,63,64]

for i in back:
    newsample.loc[newsample['PositionId'] == i, 'PositionId'] = 2

for j in mid:
    newsample.loc[newsample['PositionId'] == j, 'PositionId'] = 1
    
for k in front:
    newsample.loc[newsample['PositionId'] == k, 'PositionId'] = 0
    
newsample

K:\KJR\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
K:\KJR\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
K:\KJR\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

,SeasonId,RoundId,PositionId,Points,MatchId,EventCode
3850,2019,1,1.0,2.0,19111015,PGOK
6226,2019,1,2.0,2.0,19111015,PGOK
6635,2019,1,0.0,4.0,19111015,TRY
6652,2019,1,2.0,2.0,19111015,CVOK
7961,2019,1,0.0,4.0,19111015,TRY
...,...,...,...,...,...,...
3392810,2021,25,2.0,4.0,21111256,TRY
3392821,2021,25,1.0,2.0,21111256,CVOK
3393723,2021,25,2.0,4.0,21111256,TRY
3394249,2021,25,2.0,4.0,21111256,TRY


In [101]:
for i in range(13):
    data.loc[data['RoundId'] == i, 'RoundId'] = 1
for i in range(13, 25):
    data.loc[data['RoundId'] == i, 'RoundId'] = 0
data = data.rename({'RoundId': 'FirstSeasonHalf'}, axis = 1)

NameError: name 'data' is not defined

In [ ]:
newsample.to_csv('TimeSeriesData.csv')

In [110]:
sample.columns

Index(['SeasonId', 'RoundId', 'PositionId', 'Captain', 'Tackle', 'Points',
       'TotalPossessionSecs', '0_20_ElapsedSecs', '20_Half_ElapsedSecs',
       'Half_20_ElapsedSecs', '20_Try_ElapsedSecs', 'PBNS', 'RCVS', 'MACS',
       'MACD', 'MBCS', 'MBCD', 'OADS', 'TKED', 'MKRS', 'MKRD', 'TRCS',
       'Tackle_sum', 'Pass_sum', 'C', 'L', 'NL', 'NR', 'R', 'SL', 'SR',
       'Goal_CVOK', 'Goal_FGOK', 'Goal_PGOK', 'Goal_TRY', 'Weather_2',
       'Weather_3', 'Weather_4', 'Weather_1040', 'Half_1', 'Half_2', 'Half_3',
       'Half_4'],
      dtype='object')